# Data Source 

The model loads 12,707 molecules and their reported toxicity endpoints from the Tox21 data set. Resources and descriptions of the data set are housed here https://tox21.gov/resources/ as well as other places.

The data set includes results for commonly tested nuclear receptor assays and stress response assays, which measure how likely a molecule is to cause a problematic interaction with those classes of proteins on cell surfaces.

# Data Cleaning

The data set indexes the tested molecules by the InChIKey, sometimes referred to as a hashed InChI; a fixed length (27 character) condensed digital representation of the InChI that is not human-understandable

1) To make the data set useful, the script first converts from InChIKey to InChI, a textual identifier for chemical substances, designed to provide a standard way to encode molecular information and to facilitate the search for such information in databases and on the web
    - 201 of the entires had invalid InChIKey values and were removed from the data set (leaving 12,506 entries)

2) To allow the database to be searchable by users drawing structures, the inchi is then converted to a Simplified Molecular Input Line Entry System (SMILES), used to translate a chemical's three-dimensional structure into a string of symbols that is easily machine readable.
    - 2,500 entries could not convert to SMILES (leaving 10,192 entries). Likely due to stereochemical ambiguity that is a consequence of the loss of dimensionality when converting to a string.
    
3) Using the RDKit python package, the script then gets 208 unique molecular identifiers for each molecule and stores them in a dataframe
    - to avoid drawing conclusions from a very sparse matrix, the script drops any NaN values for each endpoint, leaving 7,377 valid data points in a sufficienty dense matrix.

# Model Development: SVM

1) data set is broken into test and train groups
2) SVM pipeline is built using the standard scaler and grid search to optimize. Probability is enabled to allow thresholding by certainty

# Metrics

-Precision: The ratio of correctly predicted positive observations to the total predicted positives. Most useful in cases like this, where the costs of false positives are very high

-Recall: The ratio of correctly predicted positive observations to all the observations in the actual class. More important when the cost of false negatives is high

-F1 Score: The weighted average of Precision and Recall. This score takes both False Positives and False Negatives into account

-ROC Curve and AUC: The Receiver Operating Characteristic (ROC) curve is a plot of the true positive rate against the false positive rate for the different possible cutoffs of a diagnostic test. The Area Under the Curve (AUC) represents the degree or measure of separability. It tells how much the model is capable of distinguishing between classes. The higher the AUC, the better the model is at predicting 0 classes as 0 and 1 classes as 1.

confusion matrices presented as usual; 
- top left: true negative (correctly safe)
- top right: false positive (incorrectly predicted toxic, meaning the user would throw out a potentially viable candidate)
- bottom left: false negative (incorrectly predicted safe, meaining the user would advance the molecule to animal trials and then it would fail)
- bottom right: true positive (correctly predicted toxic)

# SVM Performance

| Endpoint        | Precision | Recall  | F1 Score | AUC       | Confusion Matrix        |
|-----------------|-----------|---------|----------|-----------|-------------------------|
| NR.AhR          | 0.7869    | 0.5517  | 0.6486   | 0.8985    | [[1276, 26],<br>[78, 96]]  |
| NR.AR           | 0.8409    | 0.5441  | 0.6607   | 0.7531    | [[1589, 7],<br>[31, 37]]   |
| NR.AR.LBD       | 0.7872    | 0.6607  | 0.7184   | 0.9158    | [[1475, 10],<br>[19, 37]]  |
| NR.Aromatase    | 0.7045    | 0.4627  | 0.5586   | 0.8691    | [[1212, 13],<br>[36, 31]]  |
| NR.ER           | 0.7075    | 0.4076  | 0.5172   | 0.7959    | [[1168, 31],<br>[109, 75]] |
| NR.ER.LBD       | 0.7907    | 0.4304  | 0.5574   | 0.8175    | [[1482, 9],<br>[45, 34]]   |
| NR.PPAR.gamma   | 0.0       | 0.0     | 0.0      | 0.8635    | [[1442, 0],<br>[39, 0]]    |
| SR.ARE          | 0.6216    | 0.4423  | 0.5169   | 0.8318    | [[1050, 56],<br>[116, 92]] |
| SR.ATAD5        | 0.7632    | 0.3625  | 0.4915   | 0.8756    | [[1547, 9],<br>[51, 29]]   |
| SR.HSE          | 0.5882    | 0.2857  | 0.3846   | 0.8005    | [[1405, 14],<br>[50, 20]]  |
| SR.MMP          | 0.7926    | 0.6450  | 0.7112   | 0.9364    | [[1062, 39],<br>[82, 149]] |
| SR.p53          | 0.7077    | 0.4259  | 0.5318   | 0.8966    | [[1429, 19],<br>[62, 46]]  |


# Model Development: Dense Neural Network 

the SVM was ok. Despite the relatively limited data set i tried a simple dense neural network

- first layer of 128 (relu activation) with a dropout of 0.5 / second layer 64 (relu) / fed into a sigmoid output layer

## Certainty Threhold
to optimize the certainty threshold i experimented with one endpoint (NR.ER):

| Determinator Threshold | Precision | Recall  | F1 Score | ROC-AUC | Confusion Matrix         |
|------------------------|-----------|---------|----------|---------|--------------------------|
| 0.5                    | 0.7684    | 0.3967  | 0.5233   | 0.7630  | [[1177, 22],<br>[111, 73]] |
| 0.55                   | 0.7711    | 0.3478  | 0.4794   | 0.7704  | [[1180, 19],<br>[120, 64]] |
| 0.65                   | 0.8429    | 0.3207  | 0.4646   | 0.7650  | [[1188, 11],<br>[125, 59]] |
| 0.75                   | 0.9138    | 0.2880  | 0.4380   | 0.7697  | [[1194, 5],<br>[131, 53]]  |


the final threshold was thus set to 0.7, resulting in the below performance metrics of the currently deployed model

# DNN Performance

| Endpoint         | Precision | Recall  | F1 Score | ROC-AUC | Confusion Matrix         |
|------------------|-----------|---------|----------|---------|--------------------------|
| NR.AhR           | 0.8710    | 0.4655  | 0.6067   | 0.8961  | [[1290, 12],<br>[93, 81]]  |
| NR.AR            | 0.8780    | 0.5294  | 0.6606   | 0.8040  | [[1591, 5],<br>[32, 36]]   |
| NR.AR.LBD        | 0.8444    | 0.6786  | 0.7525   | 0.9106  | [[1478, 7],<br>[18, 38]]   |
| NR.Aromatase     | 0.8571    | 0.4478  | 0.5882   | 0.8666  | [[1220, 5],<br>[37, 30]]   |
| NR.ER            | 0.8182    | 0.3424  | 0.4828   | 0.7704  | [[1185, 14],<br>[121, 63]] |
| NR.ER.LBD        | 0.8667    | 0.3291  | 0.4771   | 0.8000  | [[1487, 4],<br>[53, 26]]   |
| NR.PPAR.gamma    | 0.9231    | 0.3077  | 0.4615   | 0.8018  | [[1441, 1],<br>[27, 12]]   |
| SR.ARE           | 0.7297    | 0.2596  | 0.3830   | 0.8303  | [[1086, 20],<br>[154, 54]] |
| SR.ATAD5         | 0.8485    | 0.3500  | 0.4956   | 0.8720  | [[1551, 5],<br>[52, 28]]   |
| SR.HSE           | 0.8750    | 0.2000  | 0.3256   | 0.7303  | [[1417, 2],<br>[56, 14]]   |
| SR.MMP           | 0.8839    | 0.5931  | 0.7098   | 0.9439  | [[1083, 18],<br>[94, 137]] |
| SR.p53           | 0.7895    | 0.4167  | 0.5455   | 0.9205  | [[1436, 12],<br>[63, 45]]  |


# Understanding Output

A "1" for a given endpoint indicates that a toxicologically relevant interaction is likely

A "0" indicates that the model expects the molecule to be benign relative to the given endpoint


## Endpoints Included

Nuclear Receptor Assays test the interaction of compounds with various proteins within cells that are responsible for sensing steroid and thyroid hormones and certain other molecules. Interactions with these receptors can indicate potential endocrine-disrupting effects of the compounds.

- **NR.AhR**: Nuclear Receptor Aryl hydrocarbon Receptor assay.
- **NR.AR**: Nuclear Receptor Androgen Receptor assay.
- **NR.AR.LBD**: Androgen Receptor Ligand Binding Domain assay.
- **NR.Aromatase**: Aromatase enzyme assay.
- **NR.ER**: Nuclear Receptor Estrogen Receptor assay.
- **NR.ER.LBD**: Estrogen Receptor Ligand Binding Domain assay.
- **NR.PPAR.gamma**: Nuclear Receptor Peroxisome Proliferator-Activated Receptor Gamma assay.

**Stress Response Assays** measure the response of cells to oxidative stress, heat shock, and DNA damage. They help in understanding how a compound might induce cellular stress or damage at a molecular level.

- **SR.ARE**: Antioxidant Response Element assay.
- **SR.ATAD5**: ATAD5 assay, possibly relating to a DNA damage response.
- **SR.HSE**: Heat Shock Element response assay.
- **SR.MMP**: Mitochondrial Membrane Potential assay.
- **SR.p53**: p53 assay, related to the tumor suppressor protein p53.

## Features

Input strings are decomposed into 209 unique features, grouped broadly into these categories:

- **Topological Descriptors**: Based on the molecular graph, these values describe the topology of the molecule without considering their positions in three-dimensional space. They can include path counts, cluster counts, and other graph-based metrics.

- **Geometric Descriptors**: Capture the spatial geometry of molecules, including aspects like molecular volume, shape, and surface area.

- **Electronic Descriptors**: Describe electronic properties of molecules, such as their polarizability, electron distribution, and potential for interaction with other molecules.

- **Constitutional Descriptors**: The simplest form of descriptors, including counts of certain atom types or functional groups within a molecule.

- **Molecular Property Descriptors**: Directly calculated from the molecular structure and include properties such as molecular weight.

- **Pharmacophore Features**: Describe the presence of specific pharmacophoric elements important for drug activity, such as hydrogen bond donors and acceptors, nitro groups, and oxygens.

- **E-state Indices**: Describe the electronic state and topological environment of atoms within a molecule.

- **Molecular Fragment Counts**: Count specific molecular fragments or substructures within a molecule.
